In [1]:
"""Imports"""
import time
from pynq.overlays.base import BaseOverlay
import socket
import multiprocessing
import os

base = BaseOverlay("base.bit")
btns = base.btns_gpio

In [2]:
%%microblaze base.PMODB

//PMODB GPIO config
//This cell configures the GPIO functionality for both "PYNQ0" and "PYNQ1", which use separate pins of PMODB as the buzzer port.


#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
void write_buzzer_pin(unsigned int pin, unsigned int val){
    gpio pin_out = gpio_open(pin); //declare a gpio object called pin_out. Use the open function to open it as a GPIO pin.
    gpio_set_direction(pin_out, GPIO_OUT); //Set the direction of the newly-defined pin_out gpio object to be an output.
    gpio_write(pin_out, val); //write the requested value by the function call to the pin.
}

In [3]:
"""PYNQ0 and PYNQ1 Code"""
BUZZER_SIGNAL_PIN_PYNQ0 = 0
PYNQ0_FREQ_HZ = 440
BUZZER_SIGNAL_PIN_PYNQ1 = 1
PYNQ1_FREQ_HZ = 220
PYNQ0_LISTEN_PORT = 43210
PYNQ1_LISTEN_PORT = 55555 #must be less than 65536

"""Buzzer Tone Method"""
def half_sec_buzzer_pulse(pin, freq_Hz):
    total_cyc = int(freq_Hz/2)
    #print("total_cyc={}".format(total_cyc))
    for i in range(total_cyc - 1):
        write_buzzer_pin(pin, 1)
        time.sleep(1.0/(2*freq_Hz))
        write_buzzer_pin(pin, 0)
        time.sleep(1.0/(2*freq_Hz))
        
"""Buttons Method"""
def get_buttons():
    if btns.read() == 1: #BTN0
        return 1
    elif btns.read() == 2: #BTN1
        return 2
    elif btns.read() == 4: #BTN2
        return 4
    elif btns.read() == 8: #BTN3
        return 8

#PYNQ0 server code:
def server_method_pynq0():
    sock_server_pynq0 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock_server_pynq0.bind(('0.0.0.0', PYNQ0_LISTEN_PORT)) #Putting all 0s means you can accept all kinds of network requests anywhere from any client (known routable address)
    sock_server_pynq0.listen()
    print('PYNQ0 waiting for connection')
    conn, addr = sock_server_pynq0.accept()
    print('PYNQ0 connected to client PYNQ1')
    data = "dummy" #dummy data so no error is thrown...
    while data is not None:
        data = conn.recv(1024)
        data = data.decode("utf-8") #convert to string
        #print(data)
        if data == "b": #'b' for buzz
            print("PYNQ0 buzzzzzz...")
            half_sec_buzzer_pulse(BUZZER_SIGNAL_PIN_PYNQ0, PYNQ0_FREQ_HZ)
        if data == "d": #'d' for disconnect
            data = None
            print("Server: PYNQ0, Client: PYNQ1 connection closing...")
    sock_server_pynq0.close()
    print("Server: PYNQ0, Client: PYNQ1 connection closed.")

#PYNQ0 client code:
def client_method_pynq0():
    sock_client_pynq0 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    button_state = get_buttons()
    while button_state != 1:
        button_state = get_buttons()
    sock_client_pynq0.connect(('127.0.0.1', PYNQ1_LISTEN_PORT)) #once BTN0 is pressed, connect the client to the server.
    time.sleep(1.0) #need buffer time so it allows user to take finger off button without it flying through
    while get_buttons() != 1: #wait for same button to be pressed to close the socket too
        if get_buttons() == 2: #BTN1
            sock_client_pynq0.sendall(b"b")
    sock_client_pynq0.sendall(b"d") #send disconnect code
    sock_client_pynq0.close()

#define two processes, one for client and one for server (code borrowed from the multiprocessing_example):
proc_server_PYNQ0 = multiprocessing.Process(target=server_method_pynq0) 
os.system("taskset -p -c {} {}".format(0, proc_server_PYNQ0.pid)) # taskset is an os command to pin the process to a specific CPU
proc_server_PYNQ0.start() # start the process
proc_client_PYNQ0 = multiprocessing.Process(target=client_method_pynq0) 
os.system("taskset -p -c {} {}".format(0, proc_client_PYNQ0.pid)) # taskset is an os command to pin the process to a specific CPU
proc_client_PYNQ0.start() # start the process

#PYNQ1 server code:
def server_method_pynq1():
    sock_server_pynq1 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock_server_pynq1.bind(('0.0.0.0', PYNQ1_LISTEN_PORT)) #Putting all 0s means you can accept all kinds of network requests anywhere from any client (known routable address)
    sock_server_pynq1.listen()
    print('PYNQ1 waiting for connection')
    conn, addr = sock_server_pynq1.accept()
    print('PYNQ1 connected to client PYNQ0')
    data = "dummy" #dummy data so no error is thrown...
    while data is not None:
        data = conn.recv(1024)
        data = data.decode("utf-8") #convert to string
        #print(data)
        if data == "b": #'b' for buzz
            print("PYNQ1 buzzzzzz...")
            half_sec_buzzer_pulse(BUZZER_SIGNAL_PIN_PYNQ1, PYNQ1_FREQ_HZ)
        if data == "d": #'d' for disconnect
            data = None
            print("Server: PYNQ1, Client: PYNQ0 connection closing...")
    sock_server_pynq1.close()
    print("Server: PYNQ1, Client: PYNQ0 connection closed.")

#PYNQ1 client code:
def client_method_pynq1():
    sock_client_pynq1 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    button_state = get_buttons()
    while button_state != 4:
        button_state = get_buttons()
    sock_client_pynq1.connect(('127.0.0.1', PYNQ0_LISTEN_PORT)) #once BTN2 is pressed, connect the client to the server.
    time.sleep(1.0) #need buffer time so it allows user to take finger off button without it flying through
    while get_buttons() != 4: #wait for same button to be pressed to close the socket too
        if get_buttons() == 8: #BTN3
            sock_client_pynq1.sendall(b"b")
    sock_client_pynq1.sendall(b"d") #send disconnect code
    sock_client_pynq1.close()

#define two processes, one for client and one for server (code borrowed from the multiprocessing_example):
proc_server_PYNQ1 = multiprocessing.Process(target=server_method_pynq1) 
os.system("taskset -p -c {} {}".format(0, proc_server_PYNQ1.pid)) # taskset is an os command to pin the process to a specific CPU
proc_server_PYNQ1.start() # start the process
proc_client_PYNQ1 = multiprocessing.Process(target=client_method_pynq1) 
os.system("taskset -p -c {} {}".format(0, proc_client_PYNQ1.pid)) # taskset is an os command to pin the process to a specific CPU
proc_client_PYNQ1.start() # start the process

#join processes
proc_server_PYNQ0.join() # join() simply means "wait for this [thread/process] to complete"
proc_client_PYNQ0.join()
proc_server_PYNQ1.join()
proc_client_PYNQ1.join()
print("All processes done")

PYNQ0 waiting for connection
PYNQ1 waiting for connection
PYNQ1 connected to client PYNQ0
PYNQ0 connected to client PYNQ1
PYNQ0 buzzzzzz...
PYNQ1 buzzzzzz...
PYNQ0 buzzzzzz...
PYNQ1 buzzzzzz...
PYNQ0 buzzzzzz...
PYNQ1 buzzzzzz...
PYNQ0 buzzzzzz...
Server: PYNQ1, Client: PYNQ0 connection closing...
Server: PYNQ1, Client: PYNQ0 connection closed.
Server: PYNQ0, Client: PYNQ1 connection closing...
Server: PYNQ0, Client: PYNQ1 connection closed.
All processes done
